In [1]:
from tvb.basic.profile import TvbProfile
TvbProfile.set_profile(TvbProfile.LIBRARY_PROFILE)

from tvb_nest.config import *
config = Config()
config.figures.SHOW_FLAG = True 
config.figures.SAVE_FLAG = True
config.figures.FIG_FORMAT = 'png'


2019-12-06 14:45:29,204 - INFO - numexpr.utils - NumExpr defaulting to 6 threads.


/home/docker/packages/tvb-root/scientific_library/tvb/datatypes/surfaces.py:61: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)


In [9]:
# import nest
# nest.Models()
# from pynestml.frontend.pynestml_frontend import install_nest  # to_nest, 
# import os
# nest_path = os.environ["NEST_INSTALL_DIR"]
# modules_path = os.environ["MYMODULES_BLD_DIR"]
# modules_and_models = {"iaf_cond_deco2014": ["iaf_cond_deco2014"], 
# #                       "iaf_cond_nmda_deco2014": ["iaf_cond_nmda_deco2014"], 
# #                       "iaf_cond_ampa_gaba_deco2014": ["iaf_cond_ampa_gaba_deco2014"], 
# #                       "scale_connection": ["scale_connection"], 
#                       "tvb_rate_wongwang": ["tvb_rate_ampa_gaba_wongwang", "tvb_rate_nmda_wongwang", 
#                                             "tvb_rate_redwongwang_exc", "tvb_rate_redwongwang_inh"]
#                       }
# for module, models in modules_and_models.items():
#     models_path = os.path.join(modules_path, '%s' % module)
#     # Compile models
#     install_nest(models_path, nest_path)
#     module_name = module + "module"
#     # Intall into NEST
#     nest.Install(module_name)
#     for model in models:  
#         print("\n" + model + "\n")
#         try:
#             print(nest.GetDefaults(model))
#             n = nest.Create(model, 1)
#             status=nest.GetStatus(n)
#             print("\n" + model + "\n")
#             print(status)
#         except:
#             n1 = nest.Create("iaf_cond_ampa_gaba_deco2014", 1)
#             n2 = nest.Create("iaf_cond_nmda_deco2014", 1)
#             nest.Connect(n1, n2, 
#                          syn_spec={'model': "scale_connection", "receptor_type": 3, "weight": 1.0})
#             nest.Connect(n2, n1, 
#                          syn_spec={'model': "scale_connection", "receptor_type": 2, "weight": 1.0})
#             print("\n" + str(nest.GetStatus(nest.GetConnections(n1))))
#             print("\n" + str(nest.GetStatus(nest.GetConnections(n2))))

In [10]:
# from tvb_nest.simulator_nest.nest_factory import compile_modules
# compile_modules(["iaf_cond_deco2014", "tvb_rate_wongwang" 
# # "iaf_cond_nmda_deco2014", "iaf_cond_ampa_gaba_deco2014", "scale_connection", "tvb_rate_wongwang"
#                 ])
# nest.Models()

In [11]:
# import pyNN.nest as sim; sim.setup(); sim.end(); print(sim)

In [2]:
from tvb_nest.config import CONFIGURED
from tvb.datatypes.connectivity import Connectivity
# Select the regions for the fine scale modeling with NEST spiking networks
nest_nodes_ids = []  # the indices of fine scale regions modeled with NEST
# In this example, we model parahippocampal cortices (left and right) with NEST
connectivity = Connectivity.from_file(config.DEFAULT_CONNECTIVITY_ZIP)
for id in range(connectivity.region_labels.shape[0]):
    if connectivity.region_labels[id].find("hippo") > 0:
        nest_nodes_ids.append(id)

In [3]:
# For interactive plotting:
# %matplotlib notebook  

# Otherwise:
%matplotlib inline 

from tvb_nest.examples.example import main_example
from tvb_nest.config import CONFIGURED
from tvb_nest.simulator_tvb.simulator import Simulator
from tvb_nest.simulator_tvb.models.reduced_wong_wang_exc_io_inh_i import ReducedWongWangExcIOInhI
from tvb_nest.simulator_tvb.models.wilson_cowan_constraint import WilsonCowan
from tvb_nest.simulator_tvb.models.generic_2d_oscillator import Generic2dOscillator
from tvb_nest.simulator_nest.builders.models.default_exc_io_inh_i import DefaultExcIOInhIBuilder
from tvb_nest.simulator_nest.builders.models.red_ww_exc_io_inh_i import RedWWExcIOInhIBuilder
from tvb_nest.simulator_nest.builders.models.ww_deco2014 import WWDeco2014Builder
from tvb_nest.interfaces.builders.models.red_ww_exc_io_inh_i \
    import RedWWexcIOinhIBuilder as InterfaceRedWWexcIOinhIBuilder
from tvb_nest.interfaces.builders.models.ww_deco2014 \
    import WWDeco2014Builder as InterfaceWWDeco2014Builder
from tvb_nest.interfaces.builders.models.wilson_cowan import \
    WilsonCowanBuilder as InterfaceWilsonCowanBuilder
from tvb_nest.interfaces.builders.models.generic_2d_oscillator import \
    Generic2DOscillatorBuilder as InterfaceGeneric2DOscillatorBuilder
from tvb_nest.plot.plotter import Plotter
from tvb.datatypes.connectivity import Connectivity
from tvb.simulator.monitors import Raw  # , Bold  # , EEG

# # -----------------------------Generic2dOscillator oscillatory regime-----------------------------------------------
# model = Generic2dOscillator()
# model.d = np.array([0.1])
# model.e = np.array([0.0])
# model.f = np.array([1.0])
# model.g = np.array([1.0])
# model.I = np.array([0.0])
# model.tau = np.array([1.0])
# model.alpha = np.array([1.0])
# model.beta = np.array([0.0])
# model.a = np.array([0.0])
# model.b = np.array([-1.0])
# model.c = np.array([0.0])
# model.variables_of_interest = ["V", "W"]
# # -----------------------------------Wilson Cowan oscillatory regime------------------------------------------------
# model = WilsonCowan()
# model.tau_e = np.array([8.0])
# model.tau_i = np.array([8.0])
# model.c_ee = np.array([16.0])
# model.c_ei = np.array([12.0])
# model.c_ie = np.array([15.0])
# model.c_ii = np.array([3.0])
# model.a_e = np.array([1.3])
# model.a_i = np.array([2.0])
# model.b_e = np.array([4.0])
# model.b_i = np.array([3.7])
# model.P = np.array([0.0])
# model.variables_of_interest = ["V", "W"]
main_example(ReducedWongWangExcIOInhI(), RedWWExcIOInhIBuilder, InterfaceRedWWexcIOinhIBuilder,
             nest_nodes_ids, nest_populations_order=100, connectivity=connectivity, 
             simulation_length=100.0, tvb_state_variable_type_label="State Variables",
             exclusive_nodes=True, noise_strength=0.001, config=CONFIGURED)

2019-12-06 14:45:34,084 - DEBUG - matplotlib - $HOME=/home/docker
2019-12-06 14:45:34,086 - DEBUG - matplotlib - CONFIGDIR=/home/docker/.config/matplotlib
2019-12-06 14:45:34,091 - DEBUG - matplotlib - matplotlib data path: /usr/share/matplotlib/mpl-data
2019-12-06 14:45:34,092 - DEBUG - matplotlib - loaded rc file /etc/matplotlibrc
2019-12-06 14:45:34,095 - DEBUG - matplotlib - matplotlib version 3.0.2
2019-12-06 14:45:34,096 - DEBUG - matplotlib - interactive is False
2019-12-06 14:45:34,097 - DEBUG - matplotlib - platform is linux
2019-12-06 14:45:34,098 - DEBUG - matplotlib - loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'site', 'os', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_colle

2019-12-06 14:45:34,164 - DEBUG - matplotlib - CACHEDIR=/home/docker/.cache/matplotlib
2019-12-06 14:45:34,165 - DEBUG - matplotlib.font_manager - font search path ['/usr/share/matplotlib/mpl-data/fonts/ttf', '/usr/share/matplotlib/mpl-data/fonts/afm', '/usr/share/matplotlib/mpl-data/fonts/pdfcorefonts']
2019-12-06 14:45:34,391 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/matplotlib/mpl-data/fonts/ttf/STIXSizThreeSymReg.ttf
2019-12-06 14:45:34,394 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/matplotlib/mpl-data/fonts/ttf/DejaVuSans-Bold.ttf
2019-12-06 14:45:34,400 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/matplotlib/mpl-data/fonts/ttf/STIXSizTwoSymReg.ttf
2019-12-06 14:45:34,402 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/matplotlib/mpl-data/fonts/ttf/STIXNonUniBol.ttf
2019-12-06 14:45:34,405 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/matplotlib/mpl-data/fonts/ttf/DejaVuSansMono-BoldOblique.

2019-12-06 14:45:34,528 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/fonts/truetype/ttf-bitstream-vera/VeraMono.ttf
2019-12-06 14:45:34,528 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/fonts/truetype/ttf-bitstream-vera/VeraBI.ttf
2019-12-06 14:45:34,529 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/fonts/opentype/freefont/FreeMonoOblique.otf
2019-12-06 14:45:34,530 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/fonts/truetype/lyx/stmary10.ttf
2019-12-06 14:45:34,531 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/fonts/opentype/freefont/FreeSerifBoldItalic.otf
2019-12-06 14:45:34,533 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/fonts/truetype/noto/NotoMono-Regular.ttf
2019-12-06 14:45:34,534 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/fonts/truetype/ttf-bitstream-vera/VeraMoBd.ttf
2019-12-06 14:45:34,534 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/

2019-12-06 14:45:34,754 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/matplotlib/mpl-data/fonts/afm/putri8a.afm
2019-12-06 14:45:34,759 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/matplotlib/mpl-data/fonts/pdfcorefonts/Courier-Oblique.afm
2019-12-06 14:45:34,762 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/matplotlib/mpl-data/fonts/afm/cmmi10.afm
2019-12-06 14:45:34,765 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/matplotlib/mpl-data/fonts/pdfcorefonts/Helvetica.afm
2019-12-06 14:45:34,774 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/matplotlib/mpl-data/fonts/afm/pcrb8a.afm
2019-12-06 14:45:34,778 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/matplotlib/mpl-data/fonts/afm/pncb8a.afm
2019-12-06 14:45:34,781 - DEBUG - matplotlib.font_manager - createFontDict: /usr/share/matplotlib/mpl-data/fonts/afm/phvl8a.afm
2019-12-06 14:45:34,784 - DEBUG - matplotlib.font_manager - createFontDic

2019-12-06 14:45:37,325 - WARNING - tvb.traits - Field seems mutable and has a default value. Consider using a lambda as a value factory 
   attribute tvb.simulator.models.oscillator.SupHopf.state_variable_range = Final(field_type=<class 'dict'>, default={'x': array([-5.,  5.]), 'y': array([-5.,  5.])}, required=True)
2019-12-06 14:45:37,914 - WARNING - tvb.traits - default contains values out of the declared domain. Ex 1.0 
   attribute  tvb_nest.simulator_tvb.models.reduced_wong_wang_exc_io_inh_i.ReducedWongWangExcIOInhI.lamda = NArray(label=':math:`\\lambda`', dtype=float64, default=array([1.]), dim_names=(), ndim=None, required=True)
2019-12-06 14:45:37,915 - WARNING - tvb.traits - Field seems mutable and has a default value. Consider using a lambda as a value factory 
   attribute tvb_nest.simulator_tvb.models.reduced_wong_wang_exc_io_inh_i.ReducedWongWangExcIOInhI.state_variable_boundaries = Final(field_type=<class 'dict'>, default={'S_e': array([0., 1.]), 'S_i': array([0., 1.]),

2019-12-06 14:45:39,945 - INFO - tvb_nest.simulator_nest.builders.base - system path: ['/home/docker/env/neurosci/nest_build/lib/python3.7/site-packages', '/home/docker/packages/tvb-multiscale/tvb_nest/examples/notebooks', '/home/docker/env/neurosci', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/home/docker/env/neurosci/lib/python3.7/site-packages', '/home/docker/env/neurosci/lib/python3.7/site-packages/NESTML-3.0.post0.dev0-py3.7.egg', '/home/docker/env/neurosci/lib/python3.7/site-packages/typing-3.7.4.1-py3.7.egg', '/home/docker/env/neurosci/lib/python3.7/site-packages/sympy-1.5rc1-py3.7.egg', '/home/docker/env/neurosci/lib/python3.7/site-packages/odetoolbox-1.0-py3.7.egg', '/home/docker/env/neurosci/lib/python3.7/site-packages/astropy-2.0.3-py3.7-linux-x86_64.egg', '/home/docker/env/neurosci/lib/python3.7/site-packages/antlr4_python3_runtime-4.7.2-py3.7.egg', '/home/docker/env/neurosci/lib/python3.7/site-packages/mpmath-1.1.0-py3.7.egg', '/h

2019-12-06 14:45:41,100 - INFO - tvb_nest.simulator_nest.models.devices - <class 'tvb_nest.simulator_nest.models.devices.NESTDeviceSet'> of model poisson_generator for R_e created!
2019-12-06 14:45:41,320 - INFO - tvb_nest.simulator_nest.models.devices - <class 'tvb_nest.interfaces.tvb_to_nest_device_interface.TVBtoNESTPoissonGeneratorInterface'> of model  for  created!
2019-12-06 14:45:41,320 - INFO - tvb_nest.simulator_nest.models.devices - <class 'tvb_nest.interfaces.tvb_to_nest_device_interface.TVBtoNESTPoissonGeneratorInterface'> of model  for  created!
2019-12-06 14:45:41,323 - INFO - tvb_nest.interfaces.tvb_to_nest_device_interface - <class 'tvb_nest.interfaces.tvb_to_nest_device_interface.TVBtoNESTPoissonGeneratorInterface'> of model  for  created!
2019-12-06 14:45:41,323 - INFO - tvb_nest.interfaces.tvb_to_nest_device_interface - <class 'tvb_nest.interfaces.tvb_to_nest_device_interface.TVBtoNESTPoissonGeneratorInterface'> of model  for  created!
2019-12-06 14:45:41,341 - INFO 

2019-12-06 14:46:20,299 - DEBUG - matplotlib.axes._base - update_title_pos
2019-12-06 14:46:20,464 - DEBUG - matplotlib.axes._base - update_title_pos
2019-12-06 14:46:20,600 - DEBUG - matplotlib.axes._base - update_title_pos
2019-12-06 14:46:20,744 - DEBUG - matplotlib.axes._base - update_title_pos
2019-12-06 14:46:21,618 - DEBUG - matplotlib.axes._base - update_title_pos
2019-12-06 14:46:21,654 - DEBUG - matplotlib.axes._base - update_title_pos
2019-12-06 14:46:21,782 - DEBUG - matplotlib.axes._base - update_title_pos
2019-12-06 14:46:21,797 - DEBUG - matplotlib.axes._base - update_title_pos
2019-12-06 14:46:21,964 - DEBUG - matplotlib.axes._base - update_title_pos
2019-12-06 14:46:21,981 - DEBUG - matplotlib.axes._base - update_title_pos
2019-12-06 14:46:22,076 - DEBUG - matplotlib.axes._base - update_title_pos
2019-12-06 14:46:22,088 - DEBUG - matplotlib.axes._base - update_title_pos
2019-12-06 14:46:22,257 - DEBUG - matplotlib.axes._base - update_title_pos
2019-12-06 14:46:22,273 -

(<tvb.datatypes.connectivity.Connectivity at 0x7fbe79c8aeb8>,
 [(array([  0.1,   0.2,   0.3,   0.4,   0.5,   0.6,   0.7,   0.8,   0.9,
            1. ,   1.1,   1.2,   1.3,   1.4,   1.5,   1.6,   1.7,   1.8,
            1.9,   2. ,   2.1,   2.2,   2.3,   2.4,   2.5,   2.6,   2.7,
            2.8,   2.9,   3. ,   3.1,   3.2,   3.3,   3.4,   3.5,   3.6,
            3.7,   3.8,   3.9,   4. ,   4.1,   4.2,   4.3,   4.4,   4.5,
            4.6,   4.7,   4.8,   4.9,   5. ,   5.1,   5.2,   5.3,   5.4,
            5.5,   5.6,   5.7,   5.8,   5.9,   6. ,   6.1,   6.2,   6.3,
            6.4,   6.5,   6.6,   6.7,   6.8,   6.9,   7. ,   7.1,   7.2,
            7.3,   7.4,   7.5,   7.6,   7.7,   7.8,   7.9,   8. ,   8.1,
            8.2,   8.3,   8.4,   8.5,   8.6,   8.7,   8.8,   8.9,   9. ,
            9.1,   9.2,   9.3,   9.4,   9.5,   9.6,   9.7,   9.8,   9.9,
           10. ,  10.1,  10.2,  10.3,  10.4,  10.5,  10.6,  10.7,  10.8,
           10.9,  11. ,  11.1,  11.2,  11.3,  11.4,  11.5,  11

2019-12-06 14:46:24,651 - DEBUG - matplotlib.pyplot - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [7]:
nest.Models()

NameError: name 'nest' is not defined

In [ ]:
from pynestml.frontend.pynestml_frontend import to_nest, install_nest
import os
models_path = os.path.join(config.nest.MODULES_DIR, "iaf_cond_deco2014")

In [ ]:
install_nest(models_path, os.environ["NEST_INSTALL_DIR"])

In [ ]:
import nest
nest.Models()

In [ ]:
nest.Install("iaf_cond_deco2014module")

In [ ]:
nest.Models()

In [ ]:
n = nest.Create("iaf_cond_deco2014")

In [ ]:
nest.GetStatus(n)

In [ ]:
import numpy as np
n1 = nest.Create("iaf_cond_deco2014", params={"w_E_ext": np.array([0.1, 0.1])})
n2 = nest.Create("iaf_cond_deco2014", params={"w_E_ext": np.array([0.1, 0.1])})

In [ ]:
nest.Connect(n1, n1, syn_spec={"receptor_type": 0})
nest.Connect(n2, n2, syn_spec={"receptor_type": 0})
nest.Connect(n1, n2, syn_spec={"receptor_type": 1})
nest.Connect(n2, n1, syn_spec={"receptor_type": 2})

In [ ]:
m = nest.Create("multimeter", params={"record_from": ["s_AMPA_ext"]})
nest.Connect(m, n1 + n2)

In [ ]:
nest.Simulate(10)

In [ ]:
nest.GetStatus(m, "events")

In [ ]:
nest.GetConnections(m)

In [ ]:
n